In [2]:
import torch
torch.set_num_threads(16)
torch.set_num_interop_threads(16)

In [3]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF, BPR, SANSA, BiVAECF, LightGCN, RecVAE, EASE, NGCF, VAECF, IBPR, NeuMF, HPF
from cornac.metrics import Precision, Recall, NDCG, MAP, MRR

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### 1. Import datasets

In [4]:
def stratified_ranking_split(
    df: pd.DataFrame,
    entity_field: str,
    test_size: float = 0.1,
    random_state: int | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Splits a ranking-based dataset into training and test sets while preserving the entity distribution.

    This function is useful for ranking models (e.g., next-best-offer, personalized recommendations) where each
    entity (e.g., user) has multiple interactions with different items, and stratification ensures that
    different user interaction levels are maintained in both splits.

    Parameters:
        df: The ranking dataset, where each row represents an interaction between an entity (e.g., user)
            and an item (e.g., game, offer).
        entity_field: The column representing the entity to be stratified.
        test_size: Fraction of unique entities to allocate to the test set.
        random_state: Random seed for reproducibility.

    Returns:
        DataFrames containing training and test data.

    Example:
        >>> train_validation_df, test_df = stratified_ranking_split(df, entity_field='user_id', test_size=0.1)
        >>> train_df, validation_df = stratified_ranking_split(
        ...     train_validation_df, entity_field='user_id', test_size=0.1
        ... )
        >>> print(train_df.shape, validation_df.shape, test_df.shape)
    """
    entity_interaction_counts = df[entity_field].value_counts()

    interaction_frequencies = entity_interaction_counts.value_counts()
    stratifiable_interaction_counts = interaction_frequencies[interaction_frequencies >= 2].index

    stratifiable_entities = entity_interaction_counts[
        entity_interaction_counts.isin(stratifiable_interaction_counts)
    ].index
    non_stratifiable_entities = entity_interaction_counts[
        ~entity_interaction_counts.isin(stratifiable_interaction_counts)
    ].index

    train_strat, test_strat = (
        train_test_split(
            stratifiable_entities,
            test_size=test_size,
            stratify=entity_interaction_counts[stratifiable_entities],
            random_state=random_state,
        )
        if len(stratifiable_entities) > 1
        else (stratifiable_entities, [])
    )

    if len(non_stratifiable_entities) > 1:
        train_non_strat, test_non_strat = train_test_split(
            non_stratifiable_entities,
            test_size=test_size,
            random_state=random_state,
        )
    else:
        train_non_strat = non_stratifiable_entities
        test_non_strat = []

    train_users = np.concatenate([train_strat, train_non_strat])
    test_users = np.concatenate([test_strat, test_non_strat])

    return df[df[entity_field].isin(train_users)], df[df[entity_field].isin(test_users)]

In [5]:
SEED = 123

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP(), MRR()]

In [6]:
lastfm_dataset = (
    pd.read_csv(
        "/Users/a-shyraliev/phd/rec-sys-research/collab_filtering_battlefield/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv",
        sep="\t",
        header=None,
        usecols=[0, 2, 3],
        names=['user_id', 'item_id', 'play_count'],
    )
    .loc[:, ['user_id', 'item_id', 'play_count']]
)

In [7]:
lastfm_dataset['user_id'].nunique(), lastfm_dataset['item_id'].nunique()

(358868, 292363)

In [8]:
_, lastfm_dataset_sample_str = stratified_ranking_split(
    lastfm_dataset,
    entity_field='user_id',
    test_size=0.1,
    random_state=SEED,
)

del lastfm_dataset, _

In [9]:
rs = RatioSplit(data=lastfm_dataset_sample_str.values, test_size=0.2, rating_threshold=0.0, seed=SEED)
rs.train_set.csr_matrix, rs.test_set.csr_matrix

/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/cornac/data/dataset.py:335: UserWarning: 5 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


(<35887x93612 sparse matrix of type '<class 'numpy.float64'>'
 	with 1402848 stored elements in Compressed Sparse Row format>,
 <35887x93612 sparse matrix of type '<class 'numpy.float64'>'
 	with 339585 stored elements in Compressed Sparse Row format>)

In [10]:
del lastfm_dataset_sample_str

### 1. MF hyperparams optimization

In [13]:
mf_models = [
    MF(use_bias=True, seed=SEED),
]

In [14]:
cornac.Experiment(
    eval_method=rs,
    models=mf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
   | MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-- + --- + ------ + ------- + ------------ + --------- + --------- + --------
MF | nan | 0.0003 |  0.0000 |       0.0000 |    0.0000 |    0.4127 | 143.9471



### 2. PMF hyperparams optimization

In [15]:
pmf_models = [
    PMF(seed=SEED),
]

In [16]:
cornac.Experiment(
    eval_method=rs,
    models=pmf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
    |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
PMF | 0.0001 | 0.0001 |  0.0000 |       0.0000 |    0.0000 |   12.6263 | 338.8230



### 3. BPR hyperparams optimization

In [17]:
bpr_models = [
    BPR(seed=SEED),
]

In [18]:
cornac.Experiment(
    eval_method=rs,
    models=bpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
    |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
BPR | 0.0273 | 0.1294 |  0.0424 |       0.0343 |    0.0358 |   11.6591 | 451.1734



### 4. BiVAECF hyperparams optimization

In [19]:
bivaecf_models = [
    BiVAECF(use_gpu=False, seed=SEED),
]

In [20]:
cornac.Experiment(
    eval_method=rs,
    models=bivaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
        |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
BiVAECF | 0.0637 | 0.2451 |  0.0930 |       0.0757 |    0.0796 | 7616.4112 | 398.0724



### 5. RecVAE hyperparams optimization

In [21]:
recvae_models = [
    RecVAE(n_epochs=10, use_gpu=False, seed=SEED),
]

In [22]:
cornac.Experiment(
    eval_method=rs,
    models=recvae_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
       |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------ + ------ + ------ + ------- + ------------ + --------- + --------- + --------
RecVae | 0.0763 | 0.2847 |  0.1142 |       0.0925 |    0.0979 | 6903.3554 | 813.5656



### 6. EASE hyperparams optimization

In [10]:
ease_models = [
    EASE(seed=SEED),
]

In [11]:
cornac.Experiment(
    eval_method=rs,
    models=ease_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

: 

### 7. HPF hyperparams optimization

In [11]:
hpf_models = [
    HPF(seed=SEED),
]

In [12]:
cornac.Experiment(
    eval_method=rs,
    models=hpf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

Learning...
Learning completed!

TEST:
...
    |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
HPF | 0.0422 | 0.1825 |  0.0641 |       0.0517 |    0.0538 |  370.1086 | 386.1145



### 8. IBPR hyperparams optimization

In [17]:
ibpr_models = [
    IBPR(),
]

In [ ]:
cornac.Experiment(
    eval_method=rs,
    models=ibpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

In [ ]:
## too long to train

### 9. SANSA hyperparams optimization

In [13]:
sansa_models = [
    SANSA(verbose=False, seed=SEED),
]

In [14]:
cornac.Experiment(
    eval_method=rs,
    models=sansa_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

INFO:sansa.core.factorizers:Computing incomplete Cholesky decomposition of X^TX + 1.0*I...
INFO:sansa.core.factorizers:Finding a fill-in reducing ordering (method = colamd)...
INFO:sansa.core.factorizers:Computing X^TX...
INFO:sansa.core.factorizers:
                X^TX info:
                    shape = (93612, 93612) 
                    nnz = 26274852 
                    density = 0.299831% 
                    size = 210.6 MB
                
INFO:sansa.core.factorizers:Sorting indices of A...
INFO:sansa.core.factorizers:Casting indptr of A to int64...
INFO:sansa.core.factorizers:Casting indices of A to int64...
INFO:sansa.core.factorizers:Computing approximate Cholesky decomposition (method = ICF)...
INFO:sansa.core.factorizers:Scaling columns and creating diagonal matrix D (LL^T -> L'DL'^T)...
INFO:sansa.core.inverters:Calculating initial guess using 1 step of Schultz method...
INFO:sansa.core.inverters:Calculating approximate inverse using Uniform Minimal Residual algorithm...



TEST:
...
      |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
----- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
SANSA | 0.0659 | 0.2583 |  0.1029 |       0.0832 |    0.0880 |  584.8892 | 786.6559



### 10. VAECF hyperparams optimization

In [15]:
vaecf_models = [
    VAECF(use_gpu=False, seed=SEED),
]

In [16]:
cornac.Experiment(
    eval_method=rs,
    models=vaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
      |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
----- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
VAECF | 0.0675 | 0.2632 |  0.1019 |       0.0809 |    0.0851 | 3870.0494 | 498.0539



### 2. Initialize models, here we are comparing: Biased MF, PMF, and BPR

In [ ]:
# mf = MF(k=10, max_iter=200, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)
# pmf = PMF(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123)
# bpr = BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)
# sansa_cholmod = SANSA(
#     name="SANSA (CHOLMOD)",
#     l2=500.0,
#     weight_matrix_density=1e-2,
#     compute_gramian=True,
#     factorizer_class="CHOLMOD",
#     factorizer_shift_step=1e-3,
#     factorizer_shift_multiplier=2.0,
#     inverter_scans=5,
#     inverter_finetune_steps=20,
#     use_absolute_value_scores=False,
#     verbose=False,
# )

# sansa_icf = SANSA(
#     name="SANSA (ICF)",
#     l2=10.0,
#     weight_matrix_density=1e-2,
#     compute_gramian=True,
#     factorizer_class="ICF",
#     factorizer_shift_step=1e-3,
#     factorizer_shift_multiplier=2.0,
#     inverter_scans=5,
#     inverter_finetune_steps=20,
#     use_absolute_value_scores=False,
#     verbose=False,
# )

# bivaecf = BiVAECF(
#     k=10,
#     use_gpu=False,
# )

lightgcn = LightGCN()

# recvae = RecVAE(
#     use_gpu=False,
# )

# ease = EASE()

ngcf = NGCF()

# vaecf = VAECF(
#     use_gpu=False,
# )

# ibpr = IBPR()

neumf = NeuMF()

# hpf = HPF()

models = [sansa_cholmod, sansa_icf, mf, pmf, bpr, bivaecf, recvae, ease, vaecf, ibpr, neumf, hpf]# lightgcn, ngcf]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/a-shyraliev/.pyenv/versions/3.12.2/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 7